In [1]:
import numpy as np
import matplotlib.pyplot as plt
import inspect

In [2]:
import importlib
import sys
sys.path.append('../')
from Utils.perturbationsnew import RBPIndividual, RBPIndividualNew1, RBPIndividualNew1fast, RBPIndividualNew2, zeroPerturb, noisePerturb, blurPerturb,multisamplebackgroundIdentification
from Utils.models import *
from Utils.data import perturb, generateSynthetic, generateSyntheticPerturbReal,ReadTS

print(sys.path)


['/Users/gali/anaconda3/envs/LIMEtf/lib/python310.zip', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/lib-dynload', '', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/setuptools/_vendor', '../', '../']


## Test important content removal (accuracy decrease)

### Load synthetic dataset and train CNN model 

In [3]:
x_train, y_train,x_test_original, y_test_original = ReadTS('MiddlePhalanxTW')
y_train = y_train - 3
y_test_original = y_test_original - 3
print(x_train.shape)
print(y_train)

(399, 80, 1)
[5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 2. 5. 5.
 5. 5. 5. 5. 3. 5. 2. 5. 5. 5. 5. 4. 5. 2. 5. 5. 5. 5. 5. 5. 5. 5. 2. 3.
 3. 2. 4. 5. 2. 5. 4. 5. 5. 0. 5. 5. 5. 5. 5. 1. 5. 1. 5. 2. 1. 2. 1. 5.
 2. 0. 5. 0. 3. 4. 0. 1. 2. 0. 1. 4. 1. 3. 1. 5. 4. 0. 5. 5. 0. 4. 5. 5.
 5. 0. 1. 0. 4. 0. 1. 5. 2. 0. 3. 1. 1. 1. 2. 1. 1. 5. 1. 1. 1. 5. 1. 4.
 1. 2. 5. 5. 0. 3. 1. 5. 3. 5. 5. 5. 1. 0. 5. 1. 0. 1. 2. 1. 3. 0. 1. 1.
 2. 4. 3. 1. 0. 5. 3. 5. 1. 1. 1. 5. 2. 0. 0. 5. 1. 0. 4. 5. 5. 1. 3. 0.
 0. 0. 1. 1. 5. 1. 5. 4. 5. 3. 4. 1. 0. 5. 5. 1. 0. 3. 0. 1. 2. 1. 5. 0.
 0. 2. 5. 1. 0. 1. 1. 5. 1. 5. 5. 2. 3. 0. 5. 5. 0. 1. 0. 5. 2. 5. 5. 5.
 5. 5. 5. 2. 5. 2. 0. 0. 2. 1. 5. 5. 0. 5. 5. 5. 1. 1. 5. 0. 1. 1. 5. 2.
 0. 4. 5. 1. 2. 1. 5. 4. 1. 5. 2. 5. 5. 4. 1. 5. 2. 5. 1. 5. 5. 4. 5. 3.
 1. 5. 3. 2. 2. 5. 5. 5. 3. 1. 5. 0. 5. 4. 5. 4. 1. 5. 5. 0. 3. 1. 2. 1.
 1. 2. 0. 5. 5. 0. 1. 5. 2. 4. 0. 3. 2. 5. 1. 5. 3. 1. 5. 3. 2. 5. 1. 0.
 4. 5. 1. 0. 2. 0. 5. 1. 4. 5. 2. 3. 0

In [4]:
from sklearn.preprocessing import StandardScaler
def normalize(TS):
    # 获取原始数据集的形状
    original_shape = TS.shape
    
    # 将数据集reshape成二维数组
    TS_2d = TS.reshape(original_shape[0] , original_shape[1])
    
    # 初始化StandardScaler
    scaler = StandardScaler()
    
    # 对数据集进行标准化
    TS_normalized = scaler.fit_transform(TS_2d)
    
    # 将标准化后的数据集reshape回原始形状
    TS = TS_normalized.reshape(original_shape)
    
    return TS

x_train = normalize(x_train)
x_test_original = normalize(x_test_original)
#print(x_train)

In [5]:
model = make_CNN_model(x_train.shape[1:],num_classes = 6)
trained_model, history = train_CNN_model(model, x_train, y_train, epochs=100, batch_size=32)

Epoch 1/100


2024-11-30 11:23:10.284869: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


10/10 [==============================] - 0s 16ms/step - loss: 1.7746 - sparse_categorical_accuracy: 0.3072 - val_loss: 1.7988 - val_sparse_categorical_accuracy: 0.1625
Epoch 2/100
10/10 [==============================] - 0s 7ms/step - loss: 1.4576 - sparse_categorical_accuracy: 0.5580 - val_loss: 1.7302 - val_sparse_categorical_accuracy: 0.4000
Epoch 3/100
10/10 [==============================] - 0s 8ms/step - loss: 1.3272 - sparse_categorical_accuracy: 0.5925 - val_loss: 1.6700 - val_sparse_categorical_accuracy: 0.3875
Epoch 4/100
10/10 [==============================] - 0s 7ms/step - loss: 1.2470 - sparse_categorical_accuracy: 0.5925 - val_loss: 1.6427 - val_sparse_categorical_accuracy: 0.3750
Epoch 5/100
10/10 [==============================] - 0s 7ms/step - loss: 1.1968 - sparse_categorical_accuracy: 0.6019 - val_loss: 1.6307 - val_sparse_categorical_accuracy: 0.3750
Epoch 6/100
10/10 [==============================] - 0s 7ms/step - loss: 1.1313 - sparse_categorical_accuracy: 0.633

In [6]:
test_loss, test_acc = trained_model.evaluate(x_test_original, y_test_original)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

5/5 [==============================] - 0s 2ms/step - loss: 1.3411 - sparse_categorical_accuracy: 0.5195
Test accuracy 0.5194805264472961
Test loss 1.3411329984664917


### Create perturbed test sets and calculate decrease in accuracy

In [7]:
print(y_test_original.shape)

(154,)


In [13]:
predictions_original = model.predict(x_test_original)
perturb_test_set = x_test_original.copy()
perturb_test_set = perturb_test_set.reshape(x_test_original.shape[0:2])
index_0 = 30
index_1 = 80
x_test_rbp = np.asarray([perturb('RBP',x,index_0,index_1) for x in perturb_test_set])
x_test_rbp = np.expand_dims(x_test_rbp, axis=-1)
predictions_rbp = model.predict(x_test_rbp)

bg = multisamplebackgroundIdentification(perturb_test_set)
x_test_new1 = np.asarray([perturb('RBP1fast',x,index_0,index_1, bg = bg) for x in perturb_test_set])
x_test_new1 = np.expand_dims(x_test_new1, axis=-1)
predictions_new1 = model.predict(x_test_new1)

#x_test_new2 = np.asarray([perturb('RBP2',x,400,500, global_ts = perturb_test_set) for x in perturb_test_set])
#x_test_new2 = np.expand_dims(x_test_new2, axis=-1)
#predictions_new2 = model.predict(x_test_new2)

x_test_zero = np.asarray([perturb('zero',x,index_0,index_1) for x in perturb_test_set])
x_test_zero = np.expand_dims(x_test_zero, axis=-1)
predictions_zero = model.predict(x_test_zero)

x_test_noise = np.asarray([perturb('noise',x,index_0,index_1) for x in perturb_test_set])
x_test_noise = np.expand_dims(x_test_noise, axis=-1)
predictions_noise = model.predict(x_test_noise)

x_test_blur = np.asarray([perturb('blur',x,index_0,index_1) for x in perturb_test_set])
x_test_blur = np.expand_dims(x_test_blur, axis=-1)
predictions_blur = model.predict(x_test_blur)

5/5 [==============================] - 0s 2ms/step


In [14]:
def l2norm(perturb):
    # 计算两个数组之间的差
    difference = perturb - x_test_original
    
    # 计算差的平方
    squared_difference = np.square(difference)
    
    # 计算平方和，沿着特征维度（axis=1），因为每个时间序列样本是一行
    sum_of_squares = np.sum(squared_difference, axis=1)
    
    # 计算平方和的平方根，即l2范数
    l2_norms = np.sqrt(sum_of_squares)
    
    average_l2_norm = np.mean(l2_norms)
    
    # l2_norms现在是一个形状为(467,)的数组，其中每个元素代表
    # 对应时间序列样本之间的l2范数
    print(average_l2_norm)  # 输出: (467,)

l2norm(x_test_rbp)
l2norm(x_test_new1)
l2norm(x_test_zero)
l2norm(x_test_noise)
l2norm(x_test_blur)

6.622172083897745
6.974083959769382
6.865684654800217
10.498723995457759
1.6847170080406377


In [15]:
def accuracy(predictions, labels):
    hits = 0
    for i in range(0,len(predictions)):
        if predictions[i] == labels[i]:
            hits += 1
    return hits/len(predictions)

In [16]:
original_accuracy = accuracy(np.argmax(predictions_original,axis=1), y_test_original)
rbp_accuracy = accuracy(np.argmax(predictions_rbp,axis=1), y_test_original)
new1_accuracy = accuracy(np.argmax(predictions_new1,axis=1), y_test_original)
#new2_accuracy = accuracy(np.argmax(predictions_new2,axis=1), y_test_original)
zero_accuracy = accuracy(np.argmax(predictions_zero,axis=1), y_test_original)
noise_accuracy = accuracy(np.argmax(predictions_noise,axis=1), y_test_original)
blur_accuracy = accuracy(np.argmax(predictions_blur,axis=1), y_test_original)

In [17]:
print(" Original Accuracy: " + str(original_accuracy) + "\n"+
    " RBP Accuracy: " + str(rbp_accuracy) + "\n"+
    " RBP1 Accuracy: " + str(new1_accuracy) + "\n"+
    #" RBP2 Accuracy: " + str(new2_accuracy) + "\n"+
      
    " Zero Accuracy: " + str(zero_accuracy) + "\n"+
    " Noise Accuracy: " + str(noise_accuracy) + "\n"+
    " Blur Accuracy: " + str(blur_accuracy) + "\n")

 Original Accuracy: 0.5194805194805194
 RBP Accuracy: 0.44155844155844154
 RBP1 Accuracy: 0.15584415584415584
 Zero Accuracy: 0.42857142857142855
 Noise Accuracy: 0.2597402597402597
 Blur Accuracy: 0.5064935064935064

